# 시가총액 데이터 분석 II

2021년 주식 시가충액 데이터로 finance data reader 에서 구축한 csv 파일을 병합해 사용했다.

### 코랩: 한글 폰트 설치

1. 한글 폰트 설치

```sh
#폰트 설치
! apt install fonts-nanum*
```

```sh
! apt install fontconfig
```

2. 폰트 캐시 생성

```sh
#폰트 캐시 생성
!fc-cache -fv
```

3. matplotlib에 남아있는 font 캐시 삭제

```sh
# matplotlib에 남아있는 font 캐시 삭제
!rm -rf ~/.cache/matplotlib/*
```

4. Runtime 재실행

5. matplotlib 에서 Nanum 한글 폰트 검색 확인

```python
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 목록에서 폰트 찾기
for font in fm.fontManager.ttflist:
    if 'Nanum' in font.name:
        print(font.name, font.fname)
```

### 필요 라이브러리

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
%matplotlib inline

In [ ]:
# 전역 폰트 설정 사용
# font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
# font_path = '/Users/qkboo/Library/Fonts/NanumGothic.otf'
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"

fontname = fm.FontProperties(fname=font_path, size=18).get_name()  # 폰트 패밀리 이름!
# plt.rc('font', family=fontname)
plt.rcParams["font.family"] = fontname
plt.rcParams["font.size"] = 12

# matplotlib 설정
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10,3)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["axes.formatter.limits"] = -10000, 10000

plt.title('한글 타이틀...')

# 2021 시가총액데이터

- https://financedata.github.io/marcap/

raw 데이터는 폴더에 년도별 gz 압축파일로 저장해서 병합해 사용.


`marcap-2021.csv.gz` 형식으로 저장되어 있다.


Date (날짜)를 인덱스(DatetimeIndex)로 포함하고 있으며, 컬럼 구성은 다음과 같습니다.

- Date : 날짜 (DatetimeIndex)
- Rank: 시가총액 순위 (당일)
- Code : 종목코드
- Name : 종명이름
- Open : 시가
- High : 고가
- Low : 저가
- Close : 종가
- Volume : 거래량
- Amount : 거래대금
- Changes : 전일대비
- ChangeCode: 등락 기호
- ChagesRatio : 전일대비 등락률
- Marcap : 시가총액(백만원)
- Stocks : 상장주식수
- MarketId : 시장기호
- Market : 시장
- Dept : 부서(한국거래소)

In [ ]:
df21 = pd.read_csv('../data/stocks/marcap-2021.csv.gz', dtype={'Code':str}, parse_dates=['Date'])
df21.head(10)

## 인덱스 변경

### 종목코드로 인덱스 변경

인덱스를 종목코드로 변경한다.

In [ ]:
#종목코드가 인덱스이다.
df21.set_index('Code', inplace=True)

In [ ]:
df21.index

In [ ]:
df21.head()

In [ ]:
df21.loc['005930']

날짜별 정렬

In [ ]:
df21.sort_values(by='Date', ascending=False).head(3)

In [ ]:
df21.sort_values(by='Name', ascending=True).head(3)

In [ ]:
df21.sort_values(by='Marcap', ascending=True).head(3)

In [ ]:
df21.info()

### 날짜 형식으로 인덱스 변경

 - 날짜 컬럼을 날짜형식으로 변환
 - groupby 객체 이용

In [ ]:
# 날짜형식으로 변환
# df21['Date'] = pd.to_datetime(df21['Date'])
# df21.info()

In [ ]:
df21.set_index('Date', drop=False, inplace=True)
df21.head()

In [ ]:
####
df21.loc['2021-04-01':'2021-06-30']

## 일자별 시가총액을 합산

In [ ]:
# 일자별 합산
df21.groupby('Date').sum()[:10]

In [ ]:
# 일자별 종목 합산
df21.groupby(['Date','Code']).sum()[:10]

In [ ]:
# 일자별 종목 평균
df21.groupby(['Date','Code']).mean()[:10]

In [ ]:
# 일자별 종목 평균
df21.groupby(['Date','Code']).agg(['sum','mean'])

In [ ]:
# 일자별 종목의 거래량/종가/시가총액 합산/평균 계산
df21.groupby(['Date','Code'])[['Close','Volume','Marcap']].agg(['sum','mean'])

In [ ]:
# 날짜별 거래량과 시가총액 합산
df21.groupby('Date').sum()[['Volume','Marcap']][:10]

### 일자별 시가총액 그래프

In [ ]:
# 일자별 시가총액
dailysum  = df21.groupby('Date').sum()['Marcap']

In [ ]:
# 2021년 일자별 시가총액 그래프
dailysum.plot(figsize=(10,6), grid=True)

### 특정일자의 시가총액 합

Groupby 로 그루핑한 일장의 시가총액/볼륨의 양

In [ ]:
# 날짜 그룹이용
daily = df21.groupby(['Date'])
daily.groups.keys()

In [ ]:
# 일자별 시가총액 합
daily.sum()

## 월별 거래량 합산

월별로 모든 거래량을 합산.

In [ ]:
df21.head(3)

In [ ]:
# df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df21.set_index('Date', inplace=True)
df21.head()

In [ ]:
df = df21.copy()
df.head()

In [ ]:
df.info()

날짜 인덱스 기반에서 일자를 기준으로 월별로 거래량의 합산을 계산한다.

In [ ]:
pd.pivot_table(df,
              index=df.index.day, 
               columns=df.index.month, 
               values=['Volume'], aggfunc=np.sum)

In [ ]:
# 월별 평균 값...
pd.pivot_table(df, index=df.index.day, columns=df.index.month, values=['Close'], aggfunc="mean")

## 종목별 시가총액



In [ ]:
df21

In [ ]:
#종목코드가 인덱스이다.
df21.set_index('Code', drop=False, inplace=True)

In [ ]:
df005930 = df21.loc['005930']
df005930.head(3)

In [ ]:
df005380 = df21.loc['005380']

In [ ]:
df005930[['Close', 'High']].plot()

In [ ]:
# 날자인덱스
df005930.plot(x='Date', y='Close')

In [ ]:
df005930.plot(x='Date', y=['Close', 'High'])

## 여러 종목

In [ ]:
_stocks = {
    '086790': '하나금융지주',
    '105560': 'KB금융',
    '316140': '우리금융지주',
    '055550': '신한지주',
    '005830': 'DB손해보험'
}

In [ ]:
stocks = df21[df21['Code'].isin(_stocks.keys())]
stocks.head()

In [ ]:
# 종목들의 2021/3/1 이후
stocks = df21[(df21['Code'].isin(_stocks.keys())) & (df21['Date'] > '2021-03-01')]
stocks.head()

필터링한 결과를 피벗해 날짜별 종목의 시가총액을 구한다.

In [ ]:
# date를 인덱스로 삼았으면
# stocks.reset_index()

### 일자별 종목의 주가 그래프

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

marcap_stocks = stocks.pivot_table(index='Date', columns='Code', values='Marcap')
marcap_stocks.rename(columns=_stocks, inplace=True)
marcap_stocks.head(5)

In [ ]:
marcap_stocks.plot(figsize=(12,6))

### 가격차 있는 주가 그래프

숫자 범위가 달라 차이가 클 경우... plot 시 다른 축을 표시한다.

In [ ]:
_stocks = {
  '005930': '삼성전자', 
  '005380': '현대차', 
  '005490': 'POSCO', 
  '105560': 'KB금융',
  '015760': '한국전력',
  '005830': 'DB손해보험'
}

In [ ]:
stocks = df21[df21['Code'].isin(_stocks.keys())]
stocks.head()

In [ ]:
marcap_stocks = stocks.pivot_table(index='Date', columns='Code', values='Marcap')
marcap_stocks.rename(columns=_stocks, inplace=True)
marcap_stocks.head(3)

삼성전자 시가총액이 커서 비교가 어렵다

In [ ]:
marcap_stocks.plot(figsize=(12,6))

이럴경우 별도의 `.plot( secondary_y=[])`로 축을 부가할 수 있다.

In [ ]:
marcap_stocks.plot(figsize=(12,6), secondary_y=['삼성전자'])

## 시가총액 비중(비율)

상대비교를 위해서 비중(비율)을 사용해 보자, 종목별 시가총액을 일자별 시가총액으로 나누어 해당 날짜의 시가총액 비중을 구해 보자.

In [ ]:
pd.options.display.float_format = '{:.4f}'.format #소수점 이하 4자리 출력

# 일별 상장종목시가 합계
marcap_daily = df21.groupby('Date').sum()['Marcap']
marcap_daily.head(3)

앞의 주식 

divide 메서드는 element-wise로 `/` 연산.
```python
DataFrame.divide(other, axis='columns', level=None, fill_value=None
```

In [ ]:
# 2021년 주식 데이터에서 총액으로 나눈다.
marcap_stocks_ratio = marcap_stocks.divide(marcap_daily, axis=0)
marcap_stocks_ratio

In [ ]:
marcap_stocks_ratio.plot(figsize=(12,6))

`삼성전자`와 `한국전력` 만을 그려보자

In [ ]:
marcap_stocks_ratio[['DB손해보험','KB금융']].plot(figsize=(12,6))

In [ ]:
marcap_stocks_ratio[['현대차','KB금융', '한국전력', 'POSCO']].plot(figsize=(12,7))

#### 시가총액 통계량

시가총액 비중의 통계치


In [ ]:
marcap_stocks_ratio.describe()

In [ ]:
marcap_stocks_ratio.describe().loc['std', :].sort_values(ascending=False)

표준편차가 가장 큰 것은 삼성전자, 작은 것은 KB금융이므로 시가종액 비중의 변화가 삼성전자가 가장 크고, KB금융은 가장 작았다는 것을 알 수 있다.

## 샘플 객체 삭제

In [ ]:
del df005380

In [ ]:
del df005930

In [ ]:
del df2

In [ ]:
del df21